<a href="https://colab.research.google.com/github/GodJiLee/Dacon/blob/main/%EA%B5%AC%EB%82%B4%EC%8B%9D%EB%8B%B9%20%EC%8B%9D%EC%88%98%EC%9D%B8%EC%9B%90%20%EC%98%88%EC%B8%A1/85_978212_xgboost_feature%EC%B5%9C%EC%A2%85%EB%B3%B8_%EA%B5%AC%EB%82%B4%EC%8B%9D%EB%8B%B9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **To Do List**
1. 요일 등 변수 팩터화 (0~4) 必 - get_dummies로 one-hot encoding 해보기 (O)
> https://teddylee777.github.io/scikit-learn/labelencoder-%EC%82%AC%EC%9A%A9%EB%B2%95    
> **get_dummies로 원-핫 인코딩 해주기로!**
2. LGBMRegressor, XGB Grid Search로 파라미터 최적화
> 두 모델 앙상블
3. EDA, 공휴일 전후 데이터와도 결합 지점 찾기 (O)
> https://www.kaggle.com/janged/xgb-lgb
4. 시각화 자료보고 인사이트 얻기 (O)
> 자기계발의 날, 공휴일 전후 처리 必
5. 메뉴 임베딩 결과 feature에 추가 (O)

# **모듈 임포트**

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
# 경고 제어
warnings.filterwarnings(action = 'ignore') # 경고를 출력하지 않음

In [ ]:
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder # 수치형 자료가 아닌 자료에 대해 모델 학습을 위해 정수형으로 바꿔주는 기능
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from lightgbm import LGBMRegressor # Tree 기반 학습 알고리즘 (Gradient Boosting 프레임워크)

from gensim.models import Word2Vec
from gensim.models import KeyedVectors # 워드 벡터를 저장하고 관리하는 기능, 유사성 조회
from sklearn.manifold import TSNE # PCA 기반 차원 축소의 문제를 해결하기 위한 방법 (티스니)
from sklearn.decomposition import PCA

# **데이터 불러오기**

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dacon_contest1/data/train.csv')
test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dacon_contest1/data/test.csv')

# **데이터 정제 (Dtype: object)**

### **일자, 요일은 라벨 인코딩 (숫자형으로 변환)**

In [ ]:
# 월, 일 인코딩
train['연도'] = pd.DatetimeIndex(train['일자']).year
test['연도'] = pd.DatetimeIndex(test['일자']).year

train['월'] = pd.DatetimeIndex(train['일자']).month
test['월'] = pd.DatetimeIndex(test['일자']).month

train['주'] = pd.DatetimeIndex(train['일자']).week
test['주'] = pd.DatetimeIndex(test['일자']).week

train['일'] = pd.DatetimeIndex(train['일자']).day
test['일'] = pd.DatetimeIndex(test['일자']).day

# **데이터 정제 (아이디어)**
* 휴가자, 재택근무자는 중식/석식에 참여하지 않을 것이므로 본사 정원수에서 차감한다.
* 즉, 본사정원수 - 휴가자 - 재택근무자
* 식사참여율 = 식사계 / 식사가능자수

In [ ]:
train['식사가능자수'] = train['본사정원수'] - train['본사휴가자수'] - train['현본사소속재택근무자수']
test['식사가능자수'] = test['본사정원수'] - test['본사휴가자수'] - test['현본사소속재택근무자수']

In [ ]:
# 식사가능자수 float -> int로 변경
train['식사가능자수'] = train['식사가능자수'].astype('int')
test['식사가능자수'] = test['식사가능자수'].astype('int')

In [ ]:
# 현본사소속재택근무자수 float -> int로 변경
train['현본사소속재택근무자수'] = train['현본사소속재택근무자수'].astype('int')
test['현본사소속재택근무자수'] = test['현본사소속재택근무자수'].astype('int')

In [ ]:
# 중식계/석식계 float -> int로 변경
train['중식계'] = train['중식계'].astype('int')
train['석식계'] = train['석식계'].astype('int')

In [ ]:
train['중식참여율'] = train['중식계'] / train['식사가능자수']
train['석식참여율'] = train['석식계'] / train['식사가능자수']

정제한 데이터를 정리하면,
### **요일 컬럼 처리**

### **Label Encoder로 범주형 자료 (요일, 요일(석식)) 수치형으로 바꾸기**

In [ ]:
# 수치형으로 바꾸지 않은 상태에서 encoder 적용
# cat_features = ['요일']

In [ ]:
# for feature in cat_features:
#     le = LabelEncoder()
#     le.fit(train[feature])
#     train[feature] = le.transform(train[feature])
#     test[feature] = le.transform(test[feature])

### **get_dummies로 원-핫 인코딩**
> 변수 간 관계성 삭제

In [ ]:
train = pd.get_dummies(train, columns = ['요일'])
test = pd.get_dummies(test, columns = ['요일'])

### **공휴일 전후 데이터 추가**
* 테스트 데이터에 설날 연휴, 어린이날 존재
* 연휴 전날 1로 인코딩 나머지 0

In [ ]:
# 새로운 column 생성
train['공휴일전후'] = 0
test['공휴일전후'] = 0

In [ ]:
# 한땀한땀 공휴일 지정..
train['공휴일전후'][17] = 1
train['공휴일전후'][3] = 1
train['공휴일전후'][62] = 1
# train['공휴일전후'][67] = 1
# train['공휴일전후'][82] = 1
train['공휴일전후'][131] = 1
# train['공휴일전후'][130] = 1
train['공휴일전후'][152] = 1
train['공휴일전후'][226] = 1
train['공휴일전후'][221] = 1
train['공휴일전후'][224] = 1
# train['공휴일전후'][244] = 1
train['공휴일전후'][245] = 1
# train['공휴일전후'][267] = 1
train['공휴일전후'][310] = 2
train['공휴일전후'][311] = 1
train['공휴일전후'][309] = 1
train['공휴일전후'][330] = 1
train['공휴일전후'][379] = 1
train['공휴일전후'][467] = 1
# train['공휴일전후'][469] = 1
train['공휴일전후'][470] = 1
train['공휴일전후'][502] = 2
# train['공휴일전후'][501] = 1
# train['공휴일전후'][511] = 1
train['공휴일전후'][565] = 1
train['공휴일전후'][623] = 1
train['공휴일전후'][651] = 1
# train['공휴일전후'][650] = 1
train['공휴일전후'][705] = 1
# train['공휴일전후'][707] = 1
train['공휴일전후'][709] = 1
# train['공휴일전후'][733] = 1
# train['공휴일전후'][748] = 1
# train['공휴일전후'][792] = 1
train['공휴일전후'][815] = 1
train['공휴일전후'][864] = 1
# train['공휴일전후'][863] = 1
train['공휴일전후'][950] = 1
train['공휴일전후'][951] = 1
train['공휴일전후'][953] = 1
train['공휴일전후'][954] = 1
train['공휴일전후'][955] = 1
train['공휴일전후'][971] = 2
# train['공휴일전후'][970] = 1
# train['공휴일전후'][1037] = 1
train['공휴일전후'][1038] = 1
train['공휴일전후'][1099] = 1
train['공휴일전후'][1129] = 1
# train['공휴일전후'][1128] = 1
train['공휴일전후'][1187] = 1
# train['공휴일전후'][1186] = 1

test['공휴일전후'][10] = 2
test['공휴일전후'][20] = 1

**원-핫 인코딩**

In [ ]:
# 변수간 관계성 삭제 (더미화, 가변수화)
train = pd.get_dummies(train, columns = ['공휴일전후'])
test = pd.get_dummies(test, columns = ['공휴일전후'])

In [ ]:
test['공휴일전후_0'][20] = 1
test['공휴일전후_1'][20] = 0

In [ ]:
train.head(2)

,일자,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,조식메뉴,중식메뉴,석식메뉴,중식계,석식계,연도,월,주,일,식사가능자수,중식참여율,석식참여율,요일_금,요일_목,요일_수,요일_월,요일_화,공휴일전후_0,공휴일전후_1,공휴일전후_2
0,2016-02-01,2601,50,150,238,0,모닝롤/찐빵 우유/두유/주스 계란후라이 호두죽/쌀밥 (쌀:국내산) 된장찌개 쥐...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 오징어찌개 쇠불고기 (쇠고기:호주산) 계란찜 ...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 육개장 자반고등어구이 두부조림 건파래무침 ...",1039,331,2016,2,5,1,2551,0.407291,0.129753,0,0,0,1,0,1,0,0
1,2016-02-02,2601,50,173,319,0,모닝롤/단호박샌드 우유/두유/주스 계란후라이 팥죽/쌀밥 (쌀:국내산) 호박젓국찌...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 김치찌개 가자미튀김 모둠소세지구이 마늘쫑무...","콩나물밥*양념장 (쌀,현미흑미:국내산) 어묵국 유산슬 (쇠고기:호주산) 아삭고추무...",867,560,2016,2,5,2,2551,0.339867,0.219522,0,0,0,0,1,1,0,0


In [ ]:
test.head(2)

,일자,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,조식메뉴,중식메뉴,석식메뉴,연도,월,주,일,식사가능자수,요일_금,요일_목,요일_수,요일_월,요일_화,공휴일전후_0,공휴일전후_1,공휴일전후_2
0,2021-01-27,2983,88,182,5,358,모닝롤/연유버터베이글 우유/주스 계란후라이/찐계란 단호박죽/흑미밥 우거지국 고기완자...,쌀밥/흑미밥/찰현미밥 대구지리 매운돈갈비찜 오꼬노미계란말이 상추무침 포기김치 양상추...,흑미밥 얼큰순두부찌개 쇠고기우엉볶음 버섯햄볶음 (New)아삭이고추무절임 포기김치,2021,1,4,27,2537,0,0,1,0,0,1,0,0
1,2021-01-28,2983,104,212,409,348,모닝롤/대만샌드위치 우유/주스 계란후라이/찐계란 누룽지탕/흑미밥 황태국 시래기지짐 ...,쌀밥/보리밥/찰현미밥 우렁된장찌개 오리주물럭 청양부추전 수제삼색무쌈 겉절이김치 양상...,충무김밥 우동국물 오징어무침 꽃맛살샐러드 얼갈이쌈장무침 석박지,2021,1,4,28,2531,0,1,0,0,0,1,0,0


## **Word Embedding**

In [ ]:
# Hyper Parameter
class CFG:
    emb_dim = 200

args = CFG

In [ ]:
df_all = pd.concat([train, test]) # train, test 모두 사용 (data leakage 문제)

In [ ]:
# Basic text preprocessing
def split_process(x, q):
    x_ = []
    x = x.split(' ')
    for i in x:
        if '(' in i and ':' in i and ')' in i:
            continue
        
        if '/' in i:
            x_.extend(i.split('/')) # 리스트 풀어서 할당 (append와 다른 점)
        
        else:
            x_.append(i)
    
    x_ = list(set(x_))
    x_.remove('')
    return x_

In [ ]:
# Get all combinations for training w2v (train + test)
food_combinations = []
for i in ['조식메뉴', '중식메뉴', '석식메뉴']:
    food_combinations += df_all[i].apply(lambda x: split_process(x, i)).to_list() # 조식, 중식, 석식 메뉴를 모두 메뉴 단위로 split해서 리스트로 만듦

In [ ]:
food_combinations

[['쌀밥',
  '된장찌개',
  '주스',
  '계란후라이',
  '우유',
  '찐빵',
  '포기김치',
  '호두죽',
  '모닝롤',
  '두유',
  '쥐어채무침'],
 ['팥죽',
  '시래기조림',
  '쌀밥',
  '주스',
  '계란후라이',
  '우유',
  '포기김치',
  '모닝롤',
  '단호박샌드',
  '두유',
  '호박젓국찌개'],
 ['표고버섯죽',
  '콩나물국',
  '쌀밥',
  '느타리호박볶음',
  '주스',
  '계란후라이',
  '우유',
  '베이글',
  '포기김치',
  '모닝롤',
  '두유'],
 ['토마토샌드',
  '쌀밥',
  '주스',
  '멸치볶음',
  '계란후라이',
  '우유',
  '근대국',
  '포기김치',
  '모닝롤',
  '닭죽',
  '두유'],
 ['쌀밥', '주스', '계란후라이', '쇠고기죽', '우유', '재첩국', '와플', '방풍나물', '포기김치', '모닝롤', '두유'],
 ['팬케익',
  '쌀밥',
  '주스',
  '계란후라이',
  '명엽채무침',
  '우유',
  '찐빵',
  '포기김치',
  '감자찌개',
  '두유',
  '견과류죽'],
 ['고구마죽',
  '쌀밥',
  '주스',
  '계란후라이',
  '우유',
  '포기김치',
  '모닝롤',
  '숙주나물',
  '봄동된장국',
  '두유',
  '야채샌드'],
 ['쌀밥',
  '콩조림',
  '주스',
  '계란후라이',
  '치즈프레즐',
  '우유',
  '잣죽',
  '민물새우찌개',
  '포기김치',
  '모닝롤',
  '두유'],
 ['마늘빵', '김구이', '쌀밥', '어묵국', '주스', '계란후라이', '단호박죽', '우유', '포기김치', '모닝롤', '두유'],
 ['쌀밥',
  '참치샌드',
  '주스',
  '계란후라이',
  '무생채',
  '우유',
  '북어계란국',
  '포기김치',
  '모닝롤',
  '흑임자죽',
  '두유'],
 ['주스', '계란후라이'

In [ ]:
# Train or load w2v model
TRAIN_W2V = True
try:
    model = Word2Vec.load('food_embedding.model')
    print('Model loaded')

except:
    if TRAIN_W2V:
        print('Training w2v')
        model = Word2Vec(sentences=food_combinations, size=args.emb_dim, window=7, min_count=0, workers=4, sg=0, iter=5000) # , epochs=5000
        model.save('food_embedding.model')

    else:
        print('Model loading failed. Do not train')

Training w2v


In [ ]:
# w2v demo
model.wv.most_similar('된장찌개')

[('차돌박이찌개', 0.569865882396698),
 ('민물새우찌개', 0.55401611328125),
 ('동태탕', 0.5529260635375977),
 ('감자국', 0.531561553478241),
 ('오징어국', 0.5225660800933838),
 ('김치두부국', 0.5207997560501099),
 ('소고기무국', 0.5069762468338013),
 ('고추장찌개', 0.4975338876247406),
 ('매생이굴국', 0.4967409670352936),
 ('김치국', 0.4951697885990143)]

# **Preprocess**

In [ ]:
def get_food_embedding(x):
    x_ = []
    x = x.split(' ')
    for i in x:
        if '(' in i and ':' in i and ')' in i:
            continue
        if '/' in i:
            x_.extend(i.split('/'))
        else:
            x_.append(i)
    x_ = list(set(x_))
    x_.remove('')
    vec_ = np.zeros(args.emb_dim)
    for i in x_:
        vec = model.wv.get_vector(i)
        vec_ += vec
    vec_ /= len(x_)
    return vec_

In [ ]:
# Get embedding
train['조식메뉴_embedding'] = train['조식메뉴'].apply(lambda x: get_food_embedding(x))
train['중식메뉴_embedding'] = train['중식메뉴'].apply(lambda x: get_food_embedding(x))
train['석식메뉴_embedding'] = train['석식메뉴'].apply(lambda x: get_food_embedding(x))

### **필요없는 features 버리기 (drop)**

In [ ]:
drops = ['일자', '연도', '일', '조식메뉴', '중식메뉴', '석식메뉴']

train = train.drop(drops, axis = 1) # axis = 1은 열을 의미 (열삭제)
# test = test.drop(drops, axis = 1)

In [ ]:
# X_common = X_train

### **모델에 사용할 데이터 생성**

In [ ]:
# features는 학습시킬 특성값들, labels는 추론할 대상값들
# 본사정원수 제외 (선형성 고려)
features = ['본사휴가자수', '본사출장자수', '본사시간외근무명령서승인건수', '현본사소속재택근무자수', '월', '주', '식사가능자수', '요일_월', '요일_화', '요일_수', '요일_목', '요일_금', '공휴일전후_0', '공휴일전후_1', '공휴일전후_2']
labels = ['중식계', '석식계'] # '중식참여율', '석식참여율'

In [ ]:
# lunch

# train
X_common = train[features]
y_train_lunch = train['중식계']
y_train_dinner = train['석식계']
# test
test_x = test[features]

In [ ]:
test.head(2)

,일자,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,조식메뉴,중식메뉴,석식메뉴,연도,월,주,일,식사가능자수,요일_금,요일_목,요일_수,요일_월,요일_화,공휴일전후_0,공휴일전후_1,공휴일전후_2
0,2021-01-27,2983,88,182,5,358,모닝롤/연유버터베이글 우유/주스 계란후라이/찐계란 단호박죽/흑미밥 우거지국 고기완자...,쌀밥/흑미밥/찰현미밥 대구지리 매운돈갈비찜 오꼬노미계란말이 상추무침 포기김치 양상추...,흑미밥 얼큰순두부찌개 쇠고기우엉볶음 버섯햄볶음 (New)아삭이고추무절임 포기김치,2021,1,4,27,2537,0,0,1,0,0,1,0,0
1,2021-01-28,2983,104,212,409,348,모닝롤/대만샌드위치 우유/주스 계란후라이/찐계란 누룽지탕/흑미밥 황태국 시래기지짐 ...,쌀밥/보리밥/찰현미밥 우렁된장찌개 오리주물럭 청양부추전 수제삼색무쌈 겉절이김치 양상...,충무김밥 우동국물 오징어무침 꽃맛살샐러드 얼갈이쌈장무침 석박지,2021,1,4,28,2531,0,1,0,0,0,1,0,0


In [ ]:
# 조식 포함한 버전, 포함하지 않은 버전
# ver2의 결과가 더 좋음

# emb_arr_lunch = np.array(df_train.iloc[:, 9].to_numpy().tolist()) + np.array(df_train.iloc[:, 10].to_numpy().tolist()) # Ver 1
# emb_arr_dinner = np.array(df_train.iloc[:, 9].to_numpy().tolist()) + np.array(df_train.iloc[:, 11].to_numpy().tolist()) # Ver 1
emb_arr_lunch = np.array(train.iloc[:, 21].to_numpy().tolist()) # Ver 2
emb_arr_dinner = np.array(train.iloc[:, 22].to_numpy().tolist()) # Ver 2

X_train_lunch = np.concatenate((X_common.to_numpy(), emb_arr_lunch), axis = 1)
X_train_dinner = np.concatenate((X_common.to_numpy(), emb_arr_dinner), axis = 1)

# **Modeling**

### **test set에 대해서도 embedding 생성**

In [ ]:
test['조식메뉴_embedding'] = test['조식메뉴'].apply(lambda x: get_food_embedding(x))
test['중식메뉴_embedding'] = test['중식메뉴'].apply(lambda x: get_food_embedding(x))
test['석식메뉴_embedding'] = test['석식메뉴'].apply(lambda x: get_food_embedding(x))

In [ ]:
test.drop(['조식메뉴', '중식메뉴', '석식메뉴'], axis = 1, inplace = True)

In [ ]:
X_test_common = test[features]

In [ ]:
X_test_common.head(2)

,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,월,주,식사가능자수,요일_월,요일_화,요일_수,요일_목,요일_금,공휴일전후_0,공휴일전후_1,공휴일전후_2
0,88,182,5,358,1,4,2537,0,0,1,0,0,1,0,0
1,104,212,409,348,1,4,2531,0,0,0,1,0,1,0,0


In [ ]:
# Get embedding
test_emb_arr_lunch = np.array(test.iloc[:, 20].to_numpy().tolist()) # Ver 2
test_emb_arr_dinner = np.array(test.iloc[:, 21].to_numpy().tolist()) # Ver 2

# Concat
test_lunch = np.concatenate((X_test_common.to_numpy(), test_emb_arr_lunch), axis = 1)
test_dinner = np.concatenate((X_test_common.to_numpy(), test_emb_arr_dinner), axis = 1)

### **XGBoost (eXtra Gradient Boost)**
* 트리 기반의 알고리즘의 앙상블 학습에서 각광받는 알고리즘 중 하나 
* 예측 성능, 수행 시간 면에서 뛰어남
* 과적합 규제
* 트리 가지치기 (pruning)
* 교차검증 내장
* 결손값 자체 처리

In [ ]:
from sklearn.model_selection import GridSearchCV
from xgboost import XGBRegressor

# GridSearch가 찾을 parameter 정의
params = {
    'learning_rate': [0.0, 0.1, 0.09, 0.089, 0.088, 0.086, 0.085, 0.08],
    'boosting_type': ['gbtree', 'gblinear', 'dart'], # 유형 추가해보기
    
}

# XGBRegressor 사용한 모델 (Boosting)
# 회귀 문제
lunch_r = XGBRegressor(objective='reg:squarederror')
dinner_r = XGBRegressor(objective='reg:squarederror')

# GridSearch 이용하여 파라미터 최적화
lunch_model = GridSearchCV(lunch_r, params, scoring='neg_mean_absolute_error')
dinner_model = GridSearchCV(dinner_r, params, scoring='neg_mean_absolute_error')

### **중식**
* 월/일/요일 제거 버전은 최적 parameter 바뀌고, mse 낮아짐 79 -> 80

In [ ]:
lunch_model.fit(X_train_lunch, y_train_lunch)
print(lunch_model.best_score_, lunch_model.best_params_)
lunch_model = lunch_model.best_estimator_

-80.9768105661226 {'boosting_type': 'gbtree', 'learning_rate': 0.086}


In [ ]:
y_pred = lunch_model.predict(test_lunch) # GridSearch 이용

In [ ]:
submit = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dacon_contest1/data/sample_submission.csv')
submit.head()

,일자,중식계,석식계
0,2021-01-27,0,0
1,2021-01-28,0,0
2,2021-01-29,0,0
3,2021-02-01,0,0
4,2021-02-02,0,0


In [ ]:
submit['중식계'] = y_pred

### **석식**

In [ ]:
dinner_model.fit(X_train_dinner, y_train_dinner)
print(dinner_model.best_score_, dinner_model.best_params_)
dinner_model = dinner_model.best_estimator_

-63.306544589996335 {'boosting_type': 'gbtree', 'learning_rate': 0.085}


In [ ]:
y_pred = dinner_model.predict(test_dinner)

In [ ]:
submit['석식계'] = y_pred

In [ ]:
# Inference
test_pred_lunch = lunch_model.predict(test_lunch)
test_pred_dinner = dinner_model.predict(test_dinner)

### **제출 파일 다운로드**


In [ ]:
submit.to_csv('submit_xgb_feature_최종본.csv', index=False)

from google.colab import files

files.download('submit_xgb_feature_최종본.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>